In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json

In [2]:
url = "https://understat.com/match/14659"

In [3]:
html_content = requests.get(url).text

In [4]:
soup = BeautifulSoup(html_content,'lxml')
soup

<!DOCTYPE html>
<html>
<head>
<base href="https://understat.com/"/>
<title>Liverpool 1 - 4 Manchester City (February 07 2021) | EPL | 2020/2021 | xG | Understat.com</title>
<meta charset="utf-8"/>
<meta content="Liverpool 1 - 4 Manchester City. Check out detailed player statistic, goals, assists, key passes, xG, shot map, xGplot." name="description"/>
<meta content="Liverpool, Manchester City, EPL, 2020/2021, (February 07 2021), xG, expected goals, shot map" name="Keywords"/>
<link href="apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="manifest.json" rel="manifest"/>
<link color="#5bbad5" href="safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="understat" name="apple-mobile-web-app-title"/>
<meta content="understat" name="application-name"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="no-cach

In [5]:
players = soup.find_all('div',class_='player-name-container')

In [6]:
for player in players:
    print(player.span.text)

Alisson
Trent Alexander-Arnold
Fabinho
Jordan Henderson
Andrew Robertson
Konstantinos Tsimikas
Curtis Jones
James Milner
Georginio Wijnaldum
Thiago Alcántara
Xherdan Shaqiri
Mohamed Salah
Roberto Firmino
Sadio Mané
Ederson
João Cancelo
Rúben Dias
John Stones
Oleksandr Zinchenko
Rodri
Ilkay Gündogan
Bernardo Silva
Riyad Mahrez
Gabriel Jesus
Phil Foden
Raheem Sterling


In [7]:
home_df = pd.DataFrame(columns=['Player Name','Player Position','Played Minutes','Goals','Yellow Cards'])
home = soup.find_all('div',class_='roster roster-home')
home_team = home[0].find('h3').a.text
labels = home[0].find_all('label',class_='player')
for label in labels:
    player_name = label.div.text.strip()
    player_position = label.find('span',class_='player-position').text.strip()
    played_minutes = label.find('span',class_='player-time').text.strip()
    
    
    goals = label.find_all('span',class_='player-goals')
    if(len(goals)!=0):
        goals = label.find_all('span',class_='goal')
    goals = len(goals)
    
    cards = label.find_all('span',class_='player-cards')
    cards = len(cards)
    
    
    home_df = home_df.append({'Player Name':player_name,
                              'Player Position':player_position,
                              'Played Minutes':played_minutes,
                              'Goals':goals,
                              'Yellow Cards':cards}
                             ,ignore_index=True)
    
home_df

,Player Name,Player Position,Played Minutes,Goals,Yellow Cards
0,Alisson,GK,90',0,0
1,Trent Alexander-Arnold,DR,90',0,0
2,Fabinho,DC,90',0,1
3,Jordan Henderson,DC,90',0,0
4,Andrew Robertson,DL,87',0,0
5,Konstantinos Tsimikas,Sub,3',0,0
6,Curtis Jones,MC,70',0,0
7,James Milner,Sub,20',0,0
8,Georginio Wijnaldum,MC,90',0,0
9,Thiago Alcántara,MC,70',0,1


In [8]:
away_df = pd.DataFrame(columns=['Player Name','Player Position','Played Minutes','Goals','Yellow Cards'])
away = soup.find_all('div',class_='roster roster-away')
away_team = away[0].find('h3').a.text
labels = away[0].find_all('label',class_='player')
for label in labels:
    player_name = label.div.text.strip()
    player_position = label.find('span',class_='player-position').text.strip()
    played_minutes = label.find('span',class_='player-time').text.strip()
    
    
    goals = label.find_all('span',class_='player-goals')
    if(len(goals)!=0):
        goals = label.find_all('span',class_='goal')
    goals = len(goals)
    
    cards = label.find_all('span',class_='player-cards')
    cards = len(cards)
    
    
    away_df = away_df.append({'Player Name':player_name,
                              'Player Position':player_position,
                              'Played Minutes':played_minutes,
                              'Goals':goals,
                              'Yellow Cards':cards}
                             ,ignore_index=True)
    
away_df

,Player Name,Player Position,Played Minutes,Goals,Yellow Cards
0,Ederson,GK,90',0,0
1,João Cancelo,DR,90',0,0
2,Rúben Dias,DC,90',0,1
3,John Stones,DC,90',0,0
4,Oleksandr Zinchenko,DL,90',0,0
5,Rodri,MC,90',0,0
6,Ilkay Gündogan,MC,90',2,0
7,Bernardo Silva,MC,90',0,0
8,Riyad Mahrez,FWR,74',0,0
9,Gabriel Jesus,Sub,16',0,0


In [9]:
Match_timeline = pd.DataFrame(columns =[home_team,'Time',away_team])
Match_timeline

,Liverpool,Time,Manchester City


In [10]:
timeline = soup.find_all('div',class_='timiline-container')
len(timeline)

12

In [11]:
def Find_Whats_Happend(rows): #Multiple things can happend at the same time (substitutions)
    incidents =[]
    for row in rows:
        #Yellow card ,Substition ,Goal,Missed penality ,Penality Scored
        
        incident=""
        #Get the palyers
        players = row.find_all('a',class_='player-name')
        
        #Yellow card
        i = row.find_all('i',class_='fas fa-square yellow-card')
        if(len(i)!=0):
            incident = incident + players[0].text + " (yellow)"
        
        #Substitution
        i = row.find_all('i',class_='fas fa-exchange-alt player-substitution')
        if(len(i)!=0):
            incident = incident + players[0].text +" <--> " + players[1].text + " (Substitution)"
        
        #Goal
        i = row.find_all('i',class_='fas fa-futbol')
        if(len(i)!=0):
            match_score = row.find('span',class_='match-score').text
            incident = incident + " (" + match_score + " ) " + players[0].text + " (Goal)"
        
        #Missed Penality
        i = row.find_all('i',class_='fas fa-futbol missed penalty')
        if(len(i)!=0):
            match_score = row.find('span',class_='match-score').text
            incident = incident + " (" + match_score + " ) " + players[0].text + " (Missed Penality)"
        
        #Penality Scored
        i = row.find_all('i',class_='fas fa-futbol penalty')
        if(len(i)!=0):
            match_score = row.find('span',class_='match-score').text
            incident = incident + " (" + match_score + " ) " + players[0].text + " (Penality Scored)"
        
        
        incidents.append(incident)
    return incidents   

In [12]:
for time in timeline:
    left_side = time.find('div',class_='timeline-left')
    left_rows = left_side.find_all('div',class_='timeline-row')
    left_incidents=""
    if(len(left_rows) !=0):
        left_incidents =Find_Whats_Happend(left_rows)
    
    
    incident_time= time.find('div',class_='timeline-minute')
    incident_time = incident_time.span.text
    
    right_side = time.find('div',class_='timeline-right')
    right_rows = right_side.find_all('div',class_='timeline-row')
    right_incidents=""
    if(len(right_rows) !=0):
        right_incidents =Find_Whats_Happend(right_rows)
        
    Match_timeline = Match_timeline.append({home_team:left_incidents,
                                'Time':incident_time,
                                away_team:right_incidents},
                               ignore_index=True)

In [13]:
Match_timeline

,Liverpool,Time,Manchester City
0,[Thiago Alcántara (yellow)],2',
1,,36',[ (0 - 0 ) Ilkay Gündogan (Missed Penality)]
2,,48',[ (0 - 1 ) Ilkay Gündogan (Goal)]
3,,61',[Rúben Dias (yellow)]
4,[ (1 - 1 ) Mohamed Salah (Penality Scored)],62',
5,[Fabinho (yellow)],69',
6,[Curtis Jones <--> James Milner (Substitution)...,70',
7,,72',[ (1 - 2 ) Ilkay Gündogan (Goal)]
8,,74',[Riyad Mahrez <--> Gabriel Jesus (Substitution)]
9,,75',[ (1 - 3 ) Raheem Sterling (Goal)]


In [14]:
Match_timeline['Liverpool'][6]

['Curtis Jones <--> James Milner (Substitution)',
 'Thiago Alcántara <--> Xherdan Shaqiri (Substitution)']

In [15]:
scripts = soup.find_all('script')

In [16]:
scripts

[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>,
 <script>
 	var shotsData 	= JSON.parse('\x7B\x22h\x22\x3A\x5B\x7B\x22id\x22\x3A\x22403856\x22,\x22minute\x22\x3A\x2223\x22,\x22result\x22\x3A\x22MissedShots\x22,\x22X\x22\x3A\x220.9309999847412109\x22,\x22Y\x22\x3A\x220.44\x22,\x22xG\x22\x3A\x220.11716905981302261\x22,\x22player\x22\x3A\x22Sadio\x20Man\x5Cu00e9\x22,\x22h_a\x22\x3A\x22h\x22,\x22player_id\x22\x3A\x22838\x22,\x22situation\x22\x3A\x22OpenPlay\x22,\x22season\x22\x3A\x222020\x22,\x22shotType\x22\x3A\x22Head\x22,\x22match_id\x22\x3A\x2214659\x22,\x22h_team\x22\x3A\x22Liverpool\x22,\x22a_team\x22\x3A\x22Manchester\x20City\x22,\x22h_goals\x22\x3A\x221\x22,\x22a_goals\x22\x3A\x224\x22,\x22date\x22\x3A\x222021\x2D02\x2D07\x2016\x3A30\x3A00\x22,\x22player_assisted\x22\x3A\x22Trent\x20Alexander\x2DArnold\x22,\x22lastAction\x22\x3A\x22Cross\x22\x7D,\x7B\x22id\x22\x3A\x22403857\x22,\x22minute

In [17]:
shots_data = scripts[1].string
start_index = shots_data.index("('")+2
end_index = shots_data.index("')")
json_data = shots_data[start_index:end_index]
json_data = json_data.encode('utf8').decode('unicode_escape')
#Convert string to json format
shots = json.loads(json_data)

In [18]:
shots

{'h': [{'id': '403856',
   'minute': '23',
   'result': 'MissedShots',
   'X': '0.9309999847412109',
   'Y': '0.44',
   'xG': '0.11716905981302261',
   'player': 'Sadio Mané',
   'h_a': 'h',
   'player_id': '838',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'Head',
   'match_id': '14659',
   'h_team': 'Liverpool',
   'a_team': 'Manchester City',
   'h_goals': '1',
   'a_goals': '4',
   'date': '2021-02-07 16:30:00',
   'player_assisted': 'Trent Alexander-Arnold',
   'lastAction': 'Cross'},
  {'id': '403857',
   'minute': '28',
   'result': 'SavedShot',
   'X': '0.855',
   'Y': '0.345',
   'xG': '0.04995096102356911',
   'player': 'Roberto Firmino',
   'h_a': 'h',
   'player_id': '482',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'RightFoot',
   'match_id': '14659',
   'h_team': 'Liverpool',
   'a_team': 'Manchester City',
   'h_goals': '1',
   'a_goals': '4',
   'date': '2021-02-07 16:30:00',
   'player_assisted': None,
   'lastAction': 'None'},


In [19]:
shots_home_team = shots['h']
shots_away_team = shots['a']

In [20]:
Shots_Home_team = pd.DataFrame(shots_home_team)
Shots_Home_team

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,403856,23,MissedShots,0.9309999847412109,0.44,0.11716905981302261,Sadio Mané,h,838,OpenPlay,2020,Head,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Trent Alexander-Arnold,Cross
1,403857,28,SavedShot,0.855,0.345,0.04995096102356911,Roberto Firmino,h,482,OpenPlay,2020,RightFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,None,None
2,403859,40,BlockedShot,0.815,0.35700000762939454,0.036689791828393936,Mohamed Salah,h,1250,OpenPlay,2020,LeftFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Sadio Mané,Pass
3,403860,44,BlockedShot,0.8119999694824219,0.655,0.0949530303478241,Thiago Alcántara,h,229,DirectFreekick,2020,RightFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,None,Standard
4,403861,44,SavedShot,0.9180000305175782,0.590999984741211,0.043351925909519196,Sadio Mané,h,838,FromCorner,2020,Head,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Trent Alexander-Arnold,Cross
5,403864,54,MissedShots,0.7440000152587891,0.46,0.02110275626182556,Curtis Jones,h,6665,OpenPlay,2020,LeftFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Roberto Firmino,Pass
6,403865,57,BlockedShot,0.8859999847412109,0.6609999847412109,0.05289136618375778,Curtis Jones,h,6665,OpenPlay,2020,RightFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,None,Tackle
7,403866,62,Goal,0.885,0.5,0.7611688375473022,Mohamed Salah,h,1250,Penalty,2020,LeftFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,None,Standard


In [21]:
Shots_Away_team = pd.DataFrame(shots_away_team)
Shots_Away_team

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,403855,7,MissedShots,0.7969999694824219,0.7269999694824218,0.01621885783970356,Oleksandr Zinchenko,a,2958,OpenPlay,2020,LeftFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Raheem Sterling,Pass
1,403858,36,MissedShots,0.885,0.5,0.7611688375473022,Ilkay Gündogan,a,314,Penalty,2020,RightFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,None,Standard
2,403862,48,SavedShot,0.9269999694824219,0.5709999847412109,0.4819853603839874,Phil Foden,a,6055,OpenPlay,2020,LeftFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Raheem Sterling,Pass
3,403863,48,Goal,0.9569999694824218,0.49599998474121093,0.7038310170173645,Ilkay Gündogan,a,314,OpenPlay,2020,RightFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,None,Rebound
4,403867,68,MissedShots,0.7880000305175782,0.500999984741211,0.03726019710302353,Bernardo Silva,a,3635,OpenPlay,2020,LeftFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,None,None
5,403868,72,Goal,0.9819999694824219,0.49400001525878906,0.9639761447906494,Ilkay Gündogan,a,314,OpenPlay,2020,RightFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Phil Foden,Pass
6,403869,75,Goal,0.9830000305175781,0.5220000076293946,0.8310533761978149,Raheem Sterling,a,618,OpenPlay,2020,Head,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Bernardo Silva,Chipped
7,403870,82,Goal,0.9119999694824219,0.3340000152587891,0.09570378065109253,Phil Foden,a,6055,OpenPlay,2020,LeftFoot,14659,Liverpool,Manchester City,1,4,2021-02-07 16:30:00,Gabriel Jesus,Chipped
